In [1]:
# The aim of the game is to deduce which three cards (a character, a location, and a weapon) are in the list of all cards, but
# not in any of the players' hands.

# This algorithm uses the process of elimination. It is not probability-based.

# Assumptions:
# 1. Players do not make suggestions they know to be entirely false (ie. their hand lacks at least 1 of the suggested cards).
# 2. Players only make accusations they think could be correct. Ie. their hand lacks all 3 of the accused cards.

In [2]:
# Relevant libraries
import math

In [3]:
# lists of all cards:
all_characters = ["Green","Mustard", "Orchid", "Peacock", "Plum", "Scarlett"]
all_locations = ["Ballroom", "Billiard Room", "Conservatory", "Dining Room", "Hall", "Kitchen", "Library", "Lounge", "Study"]
all_weapons = ["Candlestick", "Dagger", "Lead Pipe", "Revolver", "Rope", "Wrench"]
all_cards = all_characters + all_locations + all_weapons
num_cards = len(all_cards)

In [4]:
# Create a list of the players' names in the order in which they will play.
num_players = int(input("How many players are there?. There must be between 3 and 6, inclusive."))
ordered_player_list = []

# Enter your name as Rayn
for i in range(0, num_players):
    player_name = input("Please list the players' names in the order in which they will play. Press enter after each name.")
    ordered_player_list.append(player_name)

In [5]:
# Player class
class Player:
    def __init__(self, name):
        self.name = name
        self.characters_held = {
            'Green' : [],
            'Mustard' : [],
            'Orchid' : [],
            'Peacock' : [],
            'Plum' : [],
            'Scarlett' : []
        }
        self.locations_held = {
            'Ballroom' : [],
            'Billiard Room' : [],
            'Conservatory' : [],
            'Dining Room' : [],
            'Hall' : [],
            'Kitchen' : [],
            'Library' : [],
            'Lounge' : [],
            'Study' : []
            
        }
        self.weapons_held = {
            'Candlestick' : [],
            'Dagger' : [],
            'Lead Pipe' : [],
            'Revolver' : [],
            'Rope' : [],
            'Wrench' : []
        }
        self.cards_held = self.characters_held | self.locations_held | self.weapons_held
        


In [6]:
# Function to take in information about game events.
# The action possibilities are: own-card viewing, query (aka suggestion), clue card, and failed accusation.

most_recent_cards_queried = []
most_recent_answerer = ''
most_recent_card_secretly_revealed = ''
most_recent_public_revealer = ''
most_recent_card_publicly_revealed = ''
most_recent_accuser = ''
most_recent_accusation = []
my_own_cards = []

#Can use: 
# player_index = (go_number - 1) / num_players
# player_taking_turn = ordered_player_list[player_index]
# to determine which player's turn it is.
# Each turn, increase a turn_number variable by 1 (start it at 0), and pass to Info_intake function as the go_number parameter.

# Let player_taking_turn be the player's input name, not their general player identity.
def Info_intake(go_number, player_taking_turn):

    global most_recent_cards_queried
    global most_recent_answerer
    global most_recent_card_secretly_revealed
    global most_recent_card_publicly_revealed
    global most_recent_public_revealer
    global most_recent_accuser
    global most_recent_accusation
    global my_own_cards

    action = 'a'

    # ask for own cards if go_number == 0.
    if go_number == 0:
        for card in all_cards:
            card_possession = input('Do you have ' + card + '? Enter Y for yes, and anything else for no.')
            if card_possession == 'Y':
                my_own_cards.append(card)

    # If go_number > 0, ask for the type and result of each action (q, c, fa) occuring that turn.
    if go_number > 0:
        print("This is turn number " + str(go_number))
        print(player_taking_turn + " is playing")
        while action != 'e':
            action = input("Please state the action undertaken by " + player_taking_turn + ". Enter q for a query (suggestion), c for a clue card (ie. a public reveal), and fa for a failed accusation. Press e to end this turn")
            if action == 'q':
                queried_character = input('Please enter the queried character')
                queried_location = input('Please enter the queried location')
                queried_weapon = input('Please enter the queried weapon')
                most_recent_cards_queried = list((queried_character, queried_location, queried_weapon))
                most_recent_answerer = input('Who answered the query? If no-one answered, enter N')
                if player_taking_turn == 'Rayn':
                    most_recent_card_secretly_revealed = input('Which card was secretly revealed to you?')
            if action == 'c':
                most_recent_card_publicly_revealed = input('Which card was publicly revealed?')
                most_recent_public_revealer = input('Who publically revealed a card? Click N if no-one did')
            if action == 'fa':
                most_recent_accuser = input('Who made the accusation?')
                for i in range(3):
                    accused_card = input('Enter one of the accused cards')
                    most_recent_accusation.append(accused_card)

In [7]:
# function to reset necessary global variables used in information intake each turn.

def Info_intake_resetter():
    global most_recent_cards_queried
    global most_recent_answerer
    global most_recent_card_secretly_revealed
    global most_recent_card_publicly_revealed
    global most_recent_public_revealer
    global most_recent_accuser
    global most_recent_accusation
    
    most_recent_cards_queried = []
    most_recent_answerer = ''
    most_recent_card_secretly_revealed = ''
    most_recent_public_revealer = ''
    most_recent_card_publicly_revealed = ''
    most_recent_accuser = ''
    most_recent_accusation = []

    

In [8]:
# Player class instances set-up

if num_players >= 3:
    Player1 = Player(ordered_player_list[0])
    Player2 = Player(ordered_player_list[1])
    Player3 = Player(ordered_player_list[2])
    general_player_list = [Player1, Player2, Player3]
    name_GPidentity_dict = {
        ordered_player_list[0] : Player1,
        ordered_player_list[1] : Player2,
        ordered_player_list[2] : Player3
    }

if num_players >= 4:
    Player4 = Player(ordered_player_list[3])
    general_player_list.append(Player4)
    name_GPidentity_dict[ordered_player_list[3]] = Player4

if num_players >= 5:
    Player5 = Player(ordered_player_list[4])
    general_player_list.append(Player5)
    name_GPidentity_dict[ordered_player_list[4]] = Player5

if num_players == 6:
    Player6 = Player(ordered_player_list[5])
    general_player_list.append(Player6)
    name_GPidentity_dict[ordered_player_list[5]] = Player6

# The response order is the same as the playing order.

In [9]:
# name_GPidentity_dict gives the general player identity for a given name.
# Use player.name to find the name for a given general player identity

In [10]:
def Which_player_am_I():
    my_index = ordered_player_list.index('Rayn')
    if my_index == 0:
        my_player_identity = Player1
    if my_index == 1:
        my_player_identity = Player2
    if my_index == 2:
        my_player_identity = Player3
    if my_index == 3:
        my_player_identity = Player4
    if my_index == 4:
        my_player_identity = Player5
    if my_index == 5:
        my_player_identity = Player6
    
    return my_player_identity

In [11]:
# Function that provides i for HAL1,i
num_HAL1_identities = 0 # excludes those overridden
list_of_HAL1_identities = [] # excludes those overridden

def HAL1_identity_generator():
    global num_HAL1_identities
    global list_of_HAL1_identities

    HAL1_identifier = num_HAL1_identities + 1
    HAL1_identity = 'HAL1,' + str(HAL1_identifier)
    list_of_HAL1_identities.append(HAL1_identity)
    num_HAL1_identities += 1
    return HAL1_identity

# Function that provides j for LAL1,j
num_LAL1_identities = 0 # excludes those overridden
list_of_LAL1_identities = [] # excludes those overridden

def LAL1_identity_generator():
    global num_LAL1_identities
    global list_of_LAL1_identities

    LAL1_identifier = num_LAL1_identities + 1
    LAL1_identity = 'LAL1,' + str(LAL1_identifier)
    list_of_LAL1_identities.append(LAL1_identity)
    num_LAL1_identities += 1
    return LAL1_identity

In [12]:
# LAL1 override checker
# player argument is the general player identity, not their input name.

def LAL1_override_checker(player, three_potential_LAL1_cards):
    any_Ls = 0
    for card in three_potential_LAL1_cards:
        if player.cards_held[card] == ['L']:
            any_Ls += 1
    
    if any_Ls == 0:
        override = False
    else:
        override = True
    
    return override


# HAL1 override checker
# player argument is the general player identity, not their input name.

def HAL1_override_checker(player, three_potential_HAL1_cards):
    any_Hs = 0
    for card in three_potential_HAL1_cards:
        if player.cards_held[card] == ['H']:
            any_Hs += 1
    
    if any_Hs == 0:
        override = False
    else:
        override = True
    
    return override

In [13]:
# LAL1 incorporator. Only use if the LAL1 under consideration has not been overridden. player argument is the general player identity, not their input name.
def LAL1_incorporator(player, three_potential_LAL1_cards, LAL1_identity):
    for card in three_potential_LAL1_cards:
        if player.cards_held[card] == []:
            player.cards_held[card] = [LAL1_identity]
        elif 'A' in list(player.cards_held[card][0]):
            player.cards_held[card].append(LAL1_identity)

# HAL1 incorporator. Only use if the HAL1 under consideration has not been overridden. player argument is the general player identity, not their input name.
def HAL1_incorporator(player, three_potential_HAL1_cards, HAL1_identity):
    for card in three_potential_HAL1_cards:
        if player.cards_held[card] == []:
            player.cards_held[card] = [HAL1_identity]
        elif 'A' in list(player.cards_held[card][0]):
            player.cards_held[card].append(HAL1_identity)


In [14]:
# LAL1,j to L converter
# player argument is the general player identity, not their input name.
def LAL1_converter(player):
    for identity in list_of_LAL1_identities:
        num_that_identity = 0
        for card in all_cards:
            if identity in player.cards_held[card]:
                num_that_identity += 1
        
        if num_that_identity == 1:
            for card in all_cards:
                if identity in player.cards_held[card]:
                    L_incorporator(player, card)

# HAL1,i to H converter
# player argument is the general player identity, not their input name.
def HAL1_converter(player):
    for identity in list_of_HAL1_identities:
        num_that_identity = 0
        for card in all_cards:
            if identity in player.cards_held[card]:
                num_that_identity += 1
        
        if num_that_identity == 1:
            for card in all_cards:
                if identity in player.cards_held[card]:
                    H_incorporator(player, card)

In [15]:
# Return a list of the players who could not respond to a query
# Let questioner be the general player identity and answerer be the responder's input name
def Who_could_not_answer(questioner, answerer):
    GPs_who_could_not_answer = []

    if answerer != 'N':
        questioner_index = general_player_list.index(questioner)
        answerer_index = ordered_player_list.index(answerer)

        if answerer_index < questioner_index:
            num_players_who_could_not_answer = (num_players - 1) - questioner_index + answerer_index
            indices_which_could_not_answer = list(range((questioner_index + 1), num_players)) + list(range(0, answerer_index))
        elif answerer_index > questioner_index:
            num_players_who_could_not_answer = answerer_index - questioner_index - 1
            indices_which_could_not_answer = list(range((questioner_index + 1), answerer_index))
        
        for i in indices_which_could_not_answer:
            GPs_who_could_not_answer.append(general_player_list[i])
    
    else:
        for player in general_player_list:
            if player != questioner:
                GPs_who_could_not_answer.append(player)
    
    return GPs_who_could_not_answer

In [16]:
# Make all players who cannot answer a query lack all of the queried cards
# players_who_cannot answer is a list of GP identities
def Lack_queried_cards(players_who_cannot_answer, queried_cards):
    for player in players_who_cannot_answer:
        for card in queried_cards:
            L_incorporator(player, card)

In [17]:
# Check which HAL1 identities are present in a list
def HAL1_identities_presence_checker(list_to_be_checked):
    identities_present = []
    for identity in list_of_HAL1_identities:
        if identity in list_to_be_checked:
            identities_present.append(identity)
    
    return identities_present

# Check which LAL1 identities are present in a list
def LAL1_identities_presence_checker(list_to_be_checked):
    identities_present = []
    for identity in list_of_LAL1_identities:
        if identity in list_to_be_checked:
            identities_present.append(identity)
    
    return identities_present

In [18]:
# H incorporator
# player argument is the general player identity
def H_incorporator(player, card_to_H):
    HAL1_identities_present = HAL1_identities_presence_checker(player.cards_held[card_to_H])

    for identity in HAL1_identities_present:
        for card in all_cards:
            if identity in player.cards_held[card]:
                player.cards_held[card].remove(identity)
    
    player.cards_held[card_to_H] = ['H']

    for person in general_player_list:
        if person != player:
            L_incorporator(person, card_to_H)

# L incorporator
# player argument is the general player identity
def L_incorporator(player, card_to_L):
    
    if player.cards_held[card_to_L] != ['PR']:
        LAL1_identities_present = LAL1_identities_presence_checker(player.cards_held[card_to_L])

        for identity in LAL1_identities_present:
            for card in all_cards:
                if identity in player.cards_held[card]:
                    player.cards_held[card].remove(identity)
    
        player.cards_held[card_to_L] = ['L']

# PR incorporator
# player argument is the general player identity
def PR_incorporator(player, card_to_PR):
    
    if player.cards_held[card_to_PR] != ['H']:
        LAL1_identities_present = LAL1_identities_presence_checker(player.cards_held[card_to_PR])

        for identity in LAL1_identities_present:
            for card in all_cards:
                if identity in player.cards_held[card]:
                    player.cards_held[card].remove(identity)
    
        player.cards_held[card_to_PR] = ['PR']

In [19]:
# H / L / PR counter
# Let the player parameter be the general player identity
# Let the H_or_L_or_PR parameter be a string: either 'H', 'L', or 'PR'
def H_or_L_or_PR_counter(player, H_or_L_or_PR):
    num_H_or_L_or_PR = 0
    for card in all_cards:
        if H_or_L_or_PR in player.cards_held[card]:
            num_H_or_L_or_PR += 1
    
    return num_H_or_L_or_PR

In [20]:
# Using min. and max. no. cards per player to assign Hs and Ls
num_my_cards = len(my_own_cards)
num_cards_in_hands_of_others = num_cards - num_my_cards - 3
min_num_cards_per_other_player = math.floor(num_cards_in_hands_of_others / (num_players - 1))
max_num_cards_per_other_player = math.ceil(num_cards_in_hands_of_others / (num_players - 1))

# If the number of Ls + the number of PRs a player has = num_cards - minimum number of cards they can have, the rest of their cards are Hs
# Let the player parameter be the general player identity
# Do not apply to myself
def min_num_applier(player):
    any_Ls = H_or_L_or_PR_counter(player, 'L')
    any_PRs = H_or_L_or_PR_counter(player, 'PR')
    if any_PRs + any_Ls == (num_cards - min_num_cards_per_other_player):
        for card in all_cards:
            if player.cards_held[card] != ['L'] and player.cards_held[card] != ['PR']:
                H_incorporator(player, card)
    


# If the number of Hs = the maximum number of cards they can have, the rest of their cards are Ls or PRs
# Let the player parameter be the general player identity
# Do not apply to myself
def max_num_applier(player):
    HAL1_identities_held_by_this_player = []
    for card in all_cards:
        HAL1_identities_in_this_card = HAL1_identities_presence_checker(player.cards_held[card])
        HAL1_identities_held_by_this_player.extend(HAL1_identities_in_this_card)
    # Remove duplicates from the HAL1_identities_held_by_this_player list
    HAL1_identities_held_by_this_player = list(dict.fromkeys(HAL1_identities_held_by_this_player))

    any_HAL1_identities = len(HAL1_identities_held_by_this_player)

    any_Hs = H_or_L_or_PR_counter(player, 'H')
    
    if any_Hs == max_num_cards_per_other_player:
        for card in all_cards:
            if player.cards_held[card] != 'H':
                L_incorporator(player, card)
    elif any_Hs == (max_num_cards_per_other_player - 1) and any_HAL1_identities == 1:
        for card in all_cards:
            if player.cards_held[card] != ['H']:
                if 'A' not in list(player.cards_held[card][0]):
                    L_incorporator(player, card)

In [21]:
# Converting game events into corresponding player card possession properties (H, L, HAL1,i, LAL1,j, PR).
# Let player_taking_turn be the general player identity (not their input name)!!!
def Possession_data_extraction_and_incorporation(go_number, player_taking_turn):
    global most_recent_cards_queried # list of strings
    global most_recent_answerer # string: the name of the answerer
    global most_recent_card_secretly_revealed # string
    global most_recent_card_publicly_revealed # string
    global most_recent_public_revealer # string: the name of the revealer
    global most_recent_accuser # string: the name of the accuser
    global most_recent_accusation # list of strings
    global my_own_cards # list of strings

    me = Which_player_am_I()

    # Extracting and incorporating my own cards' possession information.
    if go_number == 0:
        for card in my_own_cards:
            me.cards_held[card] = ['H']

            # no other player has those cards.
            for player in general_player_list:
                if player != me:
                    player.cards_held[card] = ['L']
        
        # Assign Ls to all other cards
        for item in all_cards:
            if item not in my_own_cards:
                me.cards_held[item] = ['L']
    
    # Extracting and incorporating query (suggestion) information
    if len(most_recent_cards_queried) > 0:

        # Apply LAL1,i to player who made the query, in line with assumption 1.
        if LAL1_override_checker(player_taking_turn, most_recent_cards_queried) == False:
            LAL1_identity_to_be_used = LAL1_identity_generator()
            LAL1_incorporator(player_taking_turn, most_recent_cards_queried, LAL1_identity_to_be_used)
        
        # Players who could not answer the query lack all three of the queried cards
        players_who_could_not_answer = Who_could_not_answer(player_taking_turn, most_recent_answerer)
        Lack_queried_cards(players_who_could_not_answer, most_recent_cards_queried)

        # Player who responded HAL1 of the queried cards (if the responder secretly reveals a card to me, they have that card)
        if most_recent_answerer != 'N':
            answerer_player_identity = name_GPidentity_dict[most_recent_answerer]
            if player_taking_turn == me:
                H_incorporator(answerer_player_identity, most_recent_card_secretly_revealed)
            if player_taking_turn != me:
                if HAL1_override_checker(answerer_player_identity, most_recent_cards_queried) == False:
                    HAL1_identity_to_be_used = HAL1_identity_generator()
                    HAL1_incorporator(answerer_player_identity, most_recent_cards_queried, HAL1_identity_to_be_used)
    
    # Extracting and incorporating clue card (public reveal) information
    if most_recent_card_publicly_revealed != '':
        if most_recent_public_revealer != 'N':
            # Player who publicly revealed a card has that card.
            revealer_GPidentity = name_GPidentity_dict[most_recent_public_revealer]
            H_incorporator(revealer_GPidentity, most_recent_card_publicly_revealed)

            # Everyone else has had that card PRed to them
            for person in general_player_list:
                if person != revealer_GPidentity:
                    PR_incorporator(person, most_recent_card_publicly_revealed)
        
        else:
            for person in general_player_list:
                PR_incorporator(person, most_recent_card_publicly_revealed)
    
    # Extracting and incorporating failed accusation information
    if most_recent_accusation != []:
        accuser_GPidentity = name_GPidentity_dict[most_recent_accuser]
        for item in most_recent_accusation:
            L_incorporator(accuser_GPidentity, item)

In [22]:
# Determine possible murder items
cards_certainly_held = []

def Murder_item_possibilities():
    global cards_certainly_held

    for player in general_player_list:
        for card in all_cards:
            if player.cards_held[card] == ['H']:
                cards_certainly_held.append(card)
    
    murder_item_possibilities = []
    for i in all_cards:
        if i not in cards_certainly_held:
            murder_item_possibilities.append(i)
    
    return murder_item_possibilities

In [23]:
# Determine which GP identity's turn it is
def Whose_turn_is_it(go_number):
    if go_number == 0:
        player = Which_player_am_I()
    else:
        player_index = (go_number - 1) % num_players
        player = general_player_list[player_index]
    
    return player

In [24]:
turn_number = -1

while True:
    turn_number += 1
    person_taking_turn = Whose_turn_is_it(turn_number)
    Info_intake_resetter()
    Info_intake(turn_number, person_taking_turn.name)
    Possession_data_extraction_and_incorporation(turn_number, person_taking_turn)
    
    for person in general_player_list:
        HAL1_converter(person)
        LAL1_converter(person)
        min_num_applier(person)
        max_num_applier(person)

        print(person.name)
        print(person.cards_held)
        print('')
    
    possible_murder_cards = Murder_item_possibilities()
    print(possible_murder_cards)
    print('')

Kaysan
{'Green': [], 'Mustard': [], 'Orchid': [], 'Peacock': [], 'Plum': ['L'], 'Scarlett': [], 'Ballroom': [], 'Billiard Room': [], 'Conservatory': ['L'], 'Dining Room': [], 'Hall': ['L'], 'Kitchen': ['L'], 'Library': [], 'Lounge': [], 'Study': [], 'Candlestick': ['L'], 'Dagger': [], 'Lead Pipe': [], 'Revolver': ['L'], 'Rope': [], 'Wrench': []}

Rayn
{'Green': ['L'], 'Mustard': ['L'], 'Orchid': ['L'], 'Peacock': ['L'], 'Plum': ['H'], 'Scarlett': ['L'], 'Ballroom': ['L'], 'Billiard Room': ['L'], 'Conservatory': ['H'], 'Dining Room': ['L'], 'Hall': ['H'], 'Kitchen': ['H'], 'Library': ['L'], 'Lounge': ['L'], 'Study': ['L'], 'Candlestick': ['H'], 'Dagger': ['L'], 'Lead Pipe': ['L'], 'Revolver': ['H'], 'Rope': ['L'], 'Wrench': ['L']}

Nishat
{'Green': [], 'Mustard': [], 'Orchid': [], 'Peacock': [], 'Plum': ['L'], 'Scarlett': [], 'Ballroom': [], 'Billiard Room': [], 'Conservatory': ['L'], 'Dining Room': [], 'Hall': ['L'], 'Kitchen': ['L'], 'Library': [], 'Lounge': [], 'Study': [], 'Candlest

KeyboardInterrupt: Interrupted by user

In [ ]:
# Class copying testing

#class Fruit:
    #def __init__(self):
        #self.colour = 'orange'

#carrot = Fruit()

#my_favourite_vegetable = carrot

#my_favourite_vegetable.colour = 'purple'
#print(my_favourite_vegetable.colour)
#print(carrot.colour)

# This code prints two purples. Ie, my_favourite_vegetable is an alias of carrot, not a copy.


# Testing the addition of class instances to lists

#class Fruit:
    #def __init__(self):
        #self.taste = 'tasty'

#kiwi = Fruit()
#dragonfruit = Fruit()

#list_of_fruits = [kiwi, dragonfruit]

#for fruit in list_of_fruits:
    #print(fruit.taste)
# This works

#a_list = []
#print(len(a_list))
# This prints 0

#class Fruit:
    #def __init__(self):
        #self.colour = 'blue'

#Fruit1 = Fruit()
#Fruit2 = Fruit()
#Fruit3 = Fruit()

#list_of_fruits = [Fruit1, Fruit2, Fruit3]

#def fruit_colour_changer(new_colour):
    #for fruit in list_of_fruits:
        #fruit.colour = new_colour

#print('Before:')
#print(Fruit1.colour, Fruit2.colour, Fruit3.colour)

#fruit_colour_changer('yellow')

#print('After:')
#print(Fruit1.colour, Fruit2.colour, Fruit3.colour)

#fruit_colour_changer('pink')

#print('After:')
#print(Fruit1.colour, Fruit2.colour, Fruit3.colour)

# This gives: 
#Before:
#blue blue blue
#After:
#yellow yellow yellow
#After:
#pink pink pink

# Ie, class instance attributes (not the whole class instance itself) can be edited from within a function without calling global
#print(list('LAL1,1'))
# prints ['L', 'A', 'L', '1', ',', '1']

#dict_of_lists = {
    #'fruit' : ['apple', 'banana', 'clementine'],
    #'spices' : ['cinnamon', 'paprika']
#}

#print(list(dict_of_lists['fruit'][0]))
# prints ['a', 'p', 'p', 'l', 'e']

#x = list(range((1 + 1), 0))
#for i in x:
    #print(i)
# prints nothing: start value must be < stop value

#print(list(range((2+1), 5)))
#print(list(range((1 + 1), 3)) + list(range(0, 0)))
#print(list(range((1 + 1), 2)))
#print(list(range(1, 1)))
#print(list(range((2 + 1), 3)) + list(range(0, 0)))

#empty_list = []
#for item in empty_list:
    #print('There is an item')
# Nothing is printed

#if 'L' in ['LAL1,1']:
    #print('rethink')
# nothing is printed

#if 'L' in 'LAL1,1':
    #print('rethink')
# rethink is printed

#single_item_list = ['Hello']
#other_list = ['Ciao']

#other_list.extend(single_item_list)
#print(other_list)
# prints ['Ciao', 'Hello']
# .append prints ['Ciao', ['Hello']]


#a_list = ['a', 'a', 'a', 'b', 'b']
#dictionary = dict.fromkeys(a_list)
#print(dictionary)
#a_list_without_repeats = list(dictionary)
#print(a_list_without_repeats)

# prints:
#{'a': None, 'b': None}
#['a', 'b']

#x = -1
#print(x)
#x += 1
#print(x)
# prints -1, then 0

#a_list = ['a', 'b', 'c']

#if 'd' not in a_list:
    #print('This works')
# prints This works

#variable = False

#if variable == False:
    #print('yay')
# prints yay

yay
